# Youtube API Practice file

In [5]:
# import dependencies
import pandas as pd
import json
import requests
from googleapiclient.discovery import build
from config import api_key
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import re

In [6]:
youtube = build('youtube', 'v3', developerKey=api_key)

In [29]:
def channel_information_grabber(video_list, title):
    # create empty dataframe
    top_channels_df = pd.DataFrame()

    # for loop to get channel details
    for channel in video_list:
        response = youtube.channels().list(
            part=['snippet', 'statistics', 'topicDetails', 'contentDetails'],
            id=channel
        ).execute()
        # append response to dataframe
        top_channels_df = top_channels_df.append(response['items'], ignore_index=True)

    # seperate the snippet column into its own dataframe
    snippet_df = pd.json_normalize(top_channels_df['snippet'])
    snippet_df = snippet_df.drop(columns=['thumbnails.default.width', 'thumbnails.default.height', 'thumbnails.medium.url', 'thumbnails.medium.width', 'thumbnails.medium.height', 'thumbnails.high.url', 'thumbnails.high.width', 'thumbnails.high.height', 'localized.title', 'localized.description'])

    # seperate the statistics column into its own dataframe
    statistics_df = pd.json_normalize(top_channels_df['statistics'])

    # seperate the topicDetails column into its own dataframe
    topicDetails_df = pd.json_normalize(top_channels_df['topicDetails'])

    # seperate the contentDetails column into its own dataframe
    contentDetails_df = pd.json_normalize(top_channels_df['contentDetails'])

    top_channels_df = top_channels_df.merge(snippet_df, left_index=True, right_index=True)
    top_channels_df = top_channels_df.merge(statistics_df, left_index=True, right_index=True)
    top_channels_df = top_channels_df.merge(topicDetails_df, left_index=True, right_index=True)
    top_channels_df = top_channels_df.merge(contentDetails_df, left_index=True, right_index=True)
    
    # change data types
    top_channels_df['publishedAt'] = pd.to_datetime(top_channels_df['publishedAt'])
    top_channels_df['viewCount'] = top_channels_df['viewCount'].astype('int64')
    top_channels_df['subscriberCount'] = top_channels_df['subscriberCount'].astype('int64')
    top_channels_df['videoCount'] = top_channels_df['videoCount'].astype('int64')

    # Remove the url from Topic Categories
    # top_channels_df['topicCategories'] = top_channels_df['topicCategories'].str.replace('https://en.wikipedia.org/wiki/', '')
    
    # drop columns that are not needed
    top_channels_df = top_channels_df.drop(columns=['kind', 'etag', 'snippet', 'statistics', 'topicDetails', 'country', 'hiddenSubscriberCount', 'contentDetails', 'relatedPlaylists.likes'])

    # sort by view count
    top_channels_df = top_channels_df.sort_values(by='viewCount', ascending=False)

    # add 'title' column
    top_channels_df['category_title'] = title

    # export as csv
    top_channels_df.to_csv(f'{title}_df.csv', index=False)

    return top_channels_df

In [8]:
def get_50_videos(channel_id):
    # get most recent uploads
    response = youtube.channels().list(
        part=['contentDetails'],
        id=channel_id
    ).execute()

    # get the playlist id
    playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    # get the videos in the playlist
    response = youtube.playlistItems().list(
        part=['contentDetails'],
        playlistId=playlist_id,
        maxResults=50
    ).execute()

    # get the video ids
    video_ids = []
    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])

    return video_ids

In [4]:
def video_details_grabber(video_id):
    response = youtube.videos().list(part=['snippet', 'statistics', 'topicDetails', 'contentDetails'], id=video_id).execute()

    # get channelID
    try:
        channel_id = response['items'][0]['snippet']['channelId']
    except:
        channel_id = np.nan

    # get video title
    try:
        video_title = response['items'][0]['snippet']['title']
        video_title_clean = ''.join(e for e in video_title if e.isalnum() or e == ' ')
        video_title_clean = video_title_clean.replace('  ', ' ')
    except:
        video_title = 'N/A'
        video_title_clean = 'N/A'

    # get video id
    try:
        video_id = response['items'][0]['id']
    except:
        video_id = np.nan

    # get the date time the video was published
    try:
        video_published = response['items'][0]['snippet']['publishedAt']
    except:
        video_published = None

    # get number of views
    try:
        video_views = response['items'][0]['statistics']['viewCount']
    except:
        video_views = 0

    # is the video made for kids?
    try:
        video_madeforkids = response['items'][0]['status']['madeForKids']
    except:
        video_madeforkids = False

    # get video description
    try:
        video_description = response['items'][0]['snippet']['description']
    except:
        video_description = ''

    # get like count
    try:
        video_likes = response['items'][0]['statistics']['likeCount']
    except:
        video_likes = 0

    # get nummber of comments
    try:
        video_comment_count = response['items'][0]['statistics']['commentCount']
    except:
        video_comment_count = 0

    # get length of video
    try:
        video_length = response['items'][0]['contentDetails']['duration']
        video_length = video_length.replace('PT', '')
        video_length = video_length.replace('H', ':')
        video_length = video_length.replace('M', ':')
        video_length = video_length.replace('S', '')
    except:
        video_length = None

    # get video description
    try:
        video_description = response['items'][0]['snippet']['description']
    except:
        video_description = ''

    # get video tags
    try:
        video_tags = response['items'][0]['snippet']['tags']
    except:
        video_tags = ''

    # retrieve youtube video results
    # video_response = youtube.commentThreads().list(part='snippet,replies', videoId=video_id).execute()

    # create empty dataframe
    df = pd.DataFrame(columns=[
        'channel_id',
        'video_title', 'video_title_clean', 'video_id', 'published', 'video_views', 'video_madeforkids', 
        'video_likes', 'video_comment_count', 'video_length', 'video_description', 'video_tags'])

    # append video details to dataframe
    df = df.append({
        'channel_id': channel_id,
        'video_title': video_title,
        'video_title_clean': video_title_clean,
        'video_id': video_id,
        'published': video_published,
        'video_views': video_views,
        'video_madeforkids': video_madeforkids,
        'video_likes': video_likes,
        'video_comment_count': video_comment_count,
        'video_length': video_length,
        'video_description': video_description,
        'video_tags': video_tags
    }, ignore_index=True)
    
    return df

In [30]:
top_channels = ['UCbCmjCuTUZos6Inko4u57UQ', 'UC-lHJZR3Gqxm24_Vd_AJ5Yw', 'UCX6OQ3DkcsbYNE6H8uQQuVA', 'UCk8GzjMOrta8yxDcKfylJYw', 'UCJplp5SjeGSdVdwsfb9Q7lQ', 
'UCJ5v_MCY6GNUBTO8-D3XoAg', 'UCvlE5gTbOvjiolFlEm-c_Ow', 'UC295-Dw_tDNtZXFeAPAW6Aw', 'UCIwFjwMjI0y7PDBVEO9-bkQ', 'UCcdwLMPsaU2ezNSJU1nFoBQ', 
'UCRijo3ddMTht_IHyNSNXpNQ', 'UC3gNmTGu-TTbFPpfSs5kNkg', 'UCfM3zsQsOnfWNUppiycmBuw', 'UC0C-w0YjGpqDXGB8IHb662A', 'UC9CoOnJkIBMdeijd9qYoT_g', 
'UC4NALVCmcmL5ntpV0thoH6w', 'UCqECaJ8Gagnn7YCbPEzWH6g', 'UCRx3mKNUdl8QE06nEug7p6Q', 'UCiGm_E4ZwYSHV3bcW1pnSeQ', 'UC4rlAVgAK0SGk-yTfe48Qpw']

In [31]:
top_education_channels = [
'UC1zZE_kJ8rQHgLTVfobLi_g', 
'UCpVm7bg6pXKo1Pr6k5kxG9A', 
'UCX6b17PVsYBQ0ip5gyeme-Q', 
'UCsooa4yRKGN_zEE8iknghZA',
'UCXhSCMRRPyxSoyLSPFxK7VA',
'UCGi_crMdUZnrcsvkCa8pt-g',
'UCZYTClx2T1of7BRZ86-8fow',
'UCHnyfMqiRRG1u-2MsSQLbXA',
'UCYenDLnIHsoqQ6smwKXQ7Hg',
'UC4a-Gbdw7vOaccHmFo40b9g']
top_cooking_channels = [
'UCJFp8uSYCjXOMnkUyb3CQ3Q',
'UCYjk_zY-iYR8YNfJmuzd70A',
'UCpSgg_ECBj25s9moCDfSTsA',
'UCsP7Bpw36J666Fct5M8u-ZA',
'UCJHA_jMfCvEnv-3kRjTCQXw', 
'UCNbngWUqL2eqRw12yAwcICg', 
'UC8gFadPgK2r1ndqLI04Xvvw', 
'UCRxAgfYexGLlu1WHGIMUDqw', 
'UCbpMy0Fg74eXXkvxJrtEn3w', 
'UCfyehHM_eo4g5JUyWmms2LA']
top_fitness_channels = [
'UCiP6wD_tYlYLYh3agzbByWQ',
'UCIJwWYOfsCfz6PjxbONYXSg', 
'UCM1Nde-9eorUhq-teaWlgUA', 
'UCBINFWq52ShSgUFEoynfSwg', 
'UCEtMRF1ywKMc4sf3EXYyDzw', 
'UCyqR7WkL8i1b6xtSssDmW9w', 
'UCGMOauU8dOd4mv2bT3Tx57w',
'UCEQi1ZNJiw3YMRwni0OLsTQ',
'UC4GJndVHEhdmqLFBHOCi97A', 
'UCiH4auDlkM0tgn9ewT3B1Vw']
top_history_channels = [
'UC9MAhZQQd9egwWCxrwSIsJQ', 
'UClfEht64_NrzHf8Y0slKEjw', 
'UC510QYlOlKNyhy_zdQxnGYw',
'UCNIuvl7V8zACPpTmmNIqP2A', 
'UCggHoXaj8BQHIiPmOxezeWA',
'UC88lvyJe7aHZmcvzvubDFRg',
'UCodbH5mUeF-m_BsNueRDjcw',
'UCv_vLHiWVBh_FR9vbeuiY-A',
'UCx-dJoP9hFCBloY9qodykvw',
'UCHdluULl5c7bilx1x1TGzJQ']
top_science_channels = [
'UCC552Sd-3nyi_tk2BudLUzA',
'UCsXVk37bltHxD1rDPwtNM8Q', 
'UC6107grRI4m0o2-emgoDnAA', 
'UCUHW94eEFW7hkUMVaZz4eDg', 
'UC06E4Y_-ybJgBUMtXx8uNNw', 
'UCmQXOAse-VnzuXHebX5I77g', 
'UCxo8ooAqXiObjuaIy10ud0A', 
'UCvJiYiBUbw4tmpRSZT2r1Hw', 
'UCJcycnanWtyOGcz34jUlYZA',
'UC9uD-W5zQHQuAVT2GdcLCvg']
top_news_channels = [
'UCn8zNIfYAQNdrFRrr8oibKw',
'UCttspZesZIDEwwpVIgoZtWQ',
'UCfwx98Wty7LhdlkxL5PZyLA',
'UCupvZG-5ko_eiXAupbDfxWw',
'UCLXo7UDZvByw2ixzpQCufnA',
'UCE2606prvXQc_noEqKxVJXA',
'UC9k-yiEpRHMNVOnOi_aQK8w',
'UCBi2mrWuNuyYy4gbM6fU18Q',
'UC1yBKRuGpC1tSM73A0ZjYjQ',
'UC16niRr50-MSBwiO3YDb3RA']
top_music_channels = [
'UC0C-w0YjGpqDXGB8IHb662A',
'UCfM3zsQsOnfWNUppiycmBuw',
'UCYvmuw-JtVrTZQ-7Y4kd63Q',
'UCqECaJ8Gagnn7YCbPEzWH6g',
'UCb2HGwORFBo94DmRx4oLzow',
'UC9CoOnJkIBMdeijd9qYoT_g',
'UCpDJl2EmP7Oh90Vylx0dZtA',
'UCa10nxShhzNrCE1o2ZOPztg',
'UCoUM-UJ7rirJYP8CQ0EIaHA',
'UCEdvpU2pFRCVqU6yIPyTpMQ']
top_comedy_channels = [
'UCY30JRSgfhYXA6i6xX1erWg',
'UCV9_KinVpV-snHe3C3n1hvA',
'UC9gFih9rw0zNCK3ZtoKQQyA',
'UC8-Th83bH_thdKZDJCrn88g',
'UCxSz6JVYmzVhtkraHWZC7HQ',
'UCfm4y4rHF5HGrSr-qbvOwOg',
'UCPDis9pjXuqyI7RYLJ-TTSA',
'UCB0d0JLn1WcGYcwwZ87d2LA',
'UCPDXXXJj9nax0fr0Wfc048g',
'UCi9cDo6239RAzPpBZO9y5SA']
top_travel_channels = [
'UCHJuQZuzapBh-CuhRYxIZrg',
'UCyEd6QBSgat5kkC6svyjudA',
'UCdPambxHRj0kdFPNoJFM98A',
'UCXsQlHGuoWqukC9vz-uonrg',
'UCd5xLBi_QU6w7RGm5TTznyQ',
'UCGaOvAFinZ7BCN_FDmw74fQ',
'UC8hI77bH0VraIw6p2PHwivQ',
'UC_ptyMRLOsS1Uj0a34a_xCA',
'UCJsSEDFFnMFvW9JWU6XUn0Q',
'UCchgIh8Tc4sTmBfnMQ5pDdg']


In [9]:
topicId_list = {
'/m/04rlf': 'Music',
'/m/05fw6t':   "Children's music",
'/m/02mscn':   "Christian music",
'/m/0ggq0m':   "Classical music",
'/m/01lyv':   "Country",
'/m/02lkt':   "Electronic music",
'/m/0glt670':    "Hip hop music",
'/m/05rwpb':   "Independent music",
'/m/03_d0':    "Jazz",
'/m/028sqc':   "Music of Asia",
'/m/0g293':   "Music of Latin America",
'/m/064t9':    "Pop music",
'/m/06cqb':   "Reggae",
'/m/06j6l':   "Rhythm and blues",
'/m/06by7':   "Rock music",
'/m/0gywn':   "Soul music",
'/m/0bzvm2':    "Gaming",
'/m/025zzc':    "Action game",
'/m/02ntfj':    "Action-adventure game",
'/m/0b1vjn':    "Casual game",
'/m/02hygl':    "Music video game",
'/m/04q1x3q':    "Puzzle video game",
'/m/01sjng':   "Racing video game",
'/m/0403l3g':    "Role-playing video game",
'/m/021bp2':    "Simulation video game",
'/m/022dc6':    "Sports game",
'/m/03hf_rm':    "Strategy video game",
'/m/06ntj':    "Sports",
'/m/0jm_':   "American football",
'/m/018jz':    "Baseball",
'/m/018w8':    "Basketball",
'/m/01cgz':    "Boxing",
'/m/09xp_':    "Cricket",
'/m/02vx4':    "Football",
'/m/037hz':    "Golf",
'/m/03tmr':    "Ice hockey",
'/m/01h7lh':    "Mixed martial arts",
'/m/0410tth':    "Motorsport",
'/m/066wd':    "Professional wrestling",
'/m/07bs0':    "Tennis",
'/m/07_53':   "Volleyball",
'/m/02jjt':    "Entertainment",
'/m/095bb':    "Animated cartoon",
'/m/09kqc':    "Humor",
'/m/02vxn':    "Movies",
'/m/05qjc':    "Performing arts",
'/m/019_rr':    "Lifestyle",
'/m/032tl':    "Fashion",
'/m/027x7n':    "Fitness",
'/m/02wbm':    "Food",
'/m/0kt51':    "Health",
'/m/03glg':    "Hobby",
'/m/068hy':    "Pets",
'/m/041xxh':    "Physical attractiveness [Beauty]",
'/m/07c1v':    "Technology",
'/m/07bxq':    "Tourism",
'/m/07yv9':    "Vehicles",
'/m/01k8wb':    "Knowledge",
'/m/098wr':    "Society"
}

In [33]:
channel_information_grabber(top_channels, 'top_channels')
channel_information_grabber(top_news_channels, 'top_news_channels')
channel_information_grabber(top_music_channels, 'top_music_channels')
channel_information_grabber(top_comedy_channels, 'top_comedy_channels')
channel_information_grabber(top_travel_channels, 'top_travel_channels')
channel_information_grabber(top_cooking_channels, 'top_cooking_channels')
channel_information_grabber(top_education_channels, 'top_education_channels')
channel_information_grabber(top_fitness_channels, 'top_fitness_channels')
channel_information_grabber(top_history_channels, 'top_history_channels')
channel_information_grabber(top_science_channels, 'top_science_channels')



,id,title,description,customUrl,publishedAt,thumbnails.default.url,defaultLanguage,viewCount,subscriberCount,videoCount,topicIds,topicCategories,relatedPlaylists.uploads,category_title
1,UCsXVk37bltHxD1rDPwtNM8Q,Kurzgesagt – In a Nutshell,Animation videos explaining things with optimi...,@kurzgesagt,2013-07-09 20:17:20+00:00,https://yt3.ggpht.com/ytc/AMLnZu8giocL5QtOHe8h...,NaN,2129399806,19500000,166,"[/m/02vxn, /m/098wr, /m/01k8wb, /m/02jjt, /m/0...","[https://en.wikipedia.org/wiki/Film, https://e...",UUsXVk37bltHxD1rDPwtNM8Q,top_science_channels
0,UCC552Sd-3nyi_tk2BudLUzA,AsapSCIENCE,Making science make sense.\n\nCreated by:\n\nM...,@asapscience,2012-05-28 17:33:43+00:00,https://yt3.ggpht.com/ytc/AMLnZu8txaNXV0qVSyaV...,NaN,1790254768,10300000,445,"[/m/01k8wb, /m/019_rr]","[https://en.wikipedia.org/wiki/Knowledge, http...",UUC552Sd-3nyi_tk2BudLUzA,top_science_channels
2,UC6107grRI4m0o2-emgoDnAA,SmarterEveryDay,I explore the world using science. That's pre...,@smartereveryday,2006-04-04 04:12:24+00:00,https://yt3.ggpht.com/ytc/AMLnZu8B3Q1hLo0DTXMl...,NaN,1074535210,10600000,359,"[/m/019_rr, /m/03glg, /m/07c1v]",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,UU6107grRI4m0o2-emgoDnAA,top_science_channels
7,UCvJiYiBUbw4tmpRSZT2r1Hw,Science Channel,"Learn about outer space, leading scientific ex...",@sciencechannel,2006-12-04 10:29:00+00:00,https://yt3.ggpht.com/VEx_GysV1ngnO_e2l-r2fiR8...,NaN,1020971516,4400000,1908,"[/m/01k8wb, /m/019_rr, /m/0f2f9, /m/02jjt]","[https://en.wikipedia.org/wiki/Knowledge, http...",UUvJiYiBUbw4tmpRSZT2r1Hw,top_science_channels
4,UC06E4Y_-ybJgBUMtXx8uNNw,TheBackyardScientist,"Aloha YouTubers, I am The Backyard Scientist!\...",@thebackyardscientist,2013-04-01 16:41:52+00:00,https://yt3.ggpht.com/ytc/AMLnZu-pMhLCPGuSocGv...,NaN,734126790,5410000,175,"[/m/019_rr, /m/01k8wb, /m/03glg]",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,UU06E4Y_-ybJgBUMtXx8uNNw,top_science_channels
9,UC9uD-W5zQHQuAVT2GdcLCvg,Science Insider,What you want to know about science. A section...,@insiderscience,2015-01-13 21:17:08+00:00,https://yt3.ggpht.com/ytc/AMLnZu8Sc12g4Xy4hHQS...,NaN,542825326,2370000,527,"[/m/01k8wb, /m/0kt51, /m/019_rr]","[https://en.wikipedia.org/wiki/Knowledge, http...",UU9uD-W5zQHQuAVT2GdcLCvg,top_science_channels
3,UCUHW94eEFW7hkUMVaZz4eDg,minutephysics,Simply put: cool physics and other sweet scien...,@minutephysics,2011-06-20 18:20:09+00:00,https://yt3.ggpht.com/ytc/AMLnZu9e3dAiZi5wVlke...,en,513233182,5580000,268,"[/m/01k8wb, /m/019_rr]","[https://en.wikipedia.org/wiki/Knowledge, http...",UUUHW94eEFW7hkUMVaZz4eDg,top_science_channels
8,UCJcycnanWtyOGcz34jUlYZA,IncredibleScience,Experience incredible and cool science experim...,@incrediblescience,2009-10-27 18:33:19+00:00,https://yt3.ggpht.com/ytc/AMLnZu-ePytD00hE3EdC...,NaN,350138531,847000,623,"[/m/01k8wb, /m/019_rr, /m/03glg]","[https://en.wikipedia.org/wiki/Knowledge, http...",UUJcycnanWtyOGcz34jUlYZA,top_science_channels
6,UCxo8ooAqXiObjuaIy10ud0A,Beyond Science,Hi! Welcome to Beyond Science!\nThis is where ...,@beyondscience,2013-10-27 20:31:16+00:00,https://yt3.ggpht.com/ytc/AMLnZu_5eztpR9min996...,NaN,349942608,2040000,480,"[/m/01k8wb, /m/019_rr]","[https://en.wikipedia.org/wiki/Knowledge, http...",UUxo8ooAqXiObjuaIy10ud0A,top_science_channels
5,UCmQXOAse-VnzuXHebX5I77g,Charlie McDonnell,,@charlieissocoollike,2007-04-03 17:20:49+00:00,https://yt3.ggpht.com/j4diJtDd3rfqKiNyhViEuco5...,NaN,0,2120000,0,NaN,NaN,UUmQXOAse-VnzuXHebX5I77g,top_science_channels


In [12]:
# Create a list of all the channels
all_channels = top_channels + top_education_channels + top_cooking_channels + top_fitness_channels + top_history_channels + top_science_channels + top_news_channels + top_music_channels + top_comedy_channels + top_travel_channels

len(all_channels)


110

In [13]:
# get the 50 videos from each channel in all_channels
video_list = []
for channel in all_channels:
    video_list.append(get_50_videos(channel))

# flatten the list
video_list = [item for sublist in video_list for item in sublist]

# convert to csv
df = pd.DataFrame(video_list)
df.to_csv('video_list.csv', index=False)

video_list




['lmH5uqwaFq8',
 '0SY0Yn0yF9o',
 'sNyF7BvVfxs',
 'K4kqqCzF-BA',
 'gfZmvllWVwY',
 'Ay3m-zZ4KLs',
 'mbPNkDEN3Ps',
 'kSx1tS52ov4',
 'o3eMkWX8E7U',
 'o1YenjwOp-A',
 'hqFfJBOrvHw',
 'QYTYKUMOuwo',
 'biXYtxZnChQ',
 'tkTYVy5sdIg',
 '-4nWoQwSHGE',
 'zhoVe42Pxb0',
 'lkBlyggtYhA',
 '3rKIRUm5oV4',
 '3QFTTxD7SuY',
 'DqhH2OUOl1M',
 'SSe98txCv00',
 'uHJN15kVTLY',
 'zHvKlszHzCg',
 'dxjciWnjd0M',
 '3LEW8jG-Q5Y',
 'usqngNJYnug',
 '3e_KEVMUvuo',
 'NSlWgxDFbe8',
 'f_8Bce7Y_p4',
 'G_MubFgSHdI',
 '-BztdXchohU',
 'tztm8t46_hI',
 'ep2qRkyIjek',
 'GbRm5DFfl88',
 'Xxa-wbT93QU',
 'Xf4XemBwzDY',
 'dfRBIE3AE34',
 'LA2q3QwhG54',
 'FGcXy0MZ3Ys',
 'iWwDq7-TCo4',
 'j7DH5splyoY',
 'U5Inxa3jK0Y',
 'Zre4vqUAgHU',
 'hqzvHfy-Ij0',
 'LCwxpu1uV60',
 'Lig3cP174Jo',
 'uCNLiwNFvuc',
 'wwosJXWGWRY',
 'TF-RmVyoEbM',
 'FtAZUKtC0Fk',
 '8XR9OzAeDqQ',
 'LzCA5zHayyk',
 'UhjjjqmGAkI',
 'cLFJjMokld8',
 'vwfQq9IA8TA',
 'P8XP4oamvzk',
 'sIw9GwePGKg',
 'KASLwYxrILs',
 'DYdrFALAlZQ',
 'JDsuYy1qsiA',
 'BQ35bZAPUnk',
 'fJ2_06vAZkw',
 'qQiwgE

In [14]:
len(video_list)

5420

In [15]:
# get information about each video
# create empty dataframe
# create empty dataframe
df = pd.DataFrame(columns=[
    'channel_id',
    'video_title', 'video_title_clean', 'video_id', 'published', 'video_views', 'video_madeforkids', 
    'video_likes', 'video_comment_count', 'video_length', 'video_description', 'video_tags'])

# get information about each video
for video in video_list:
    video_df = video_df.append(video_details_grabber(video), ignore_index=True)

video_df





,video_id,channel_id,title,description,publishedAt,categoryId,viewCount,likeCount,dislikeCount,commentCount,...,video_title,video_title_clean,published,video_views,video_madeforkids,video_likes,video_comment_count,video_length,video_description,video_tags
0,lmH5uqwaFq8,UCbCmjCuTUZos6Inko4u57UQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Airplane Song | CoComelon Nursery Rhymes & Kid...,Airplane Song CoComelon Nursery Rhymes Kids Songs,2022-11-01T07:00:15Z,3425275,False,19653,0,2:59,JJ wants to learn about airplanes and Nina's d...,"[cocomelon, abckidtv, nursery rhymes, children..."
1,0SY0Yn0yF9o,UCbCmjCuTUZos6Inko4u57UQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Wheels On The Bus + More Nursery Rhymes & Kids...,Wheels On The Bus More Nursery Rhymes Kids Son...,2022-10-29T07:00:00Z,2882582,False,15076,0,29:52,We are introduced to the magic of Lellobee Far...,"[cocomelon, abckidtv, nursery rhymes, children..."
2,sNyF7BvVfxs,UCbCmjCuTUZos6Inko4u57UQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Bingo's Bath Song | CoComelon Nursery Rhymes &...,Bingos Bath Song CoComelon Nursery Rhymes Kids...,2022-10-25T07:00:16Z,8673081,False,47763,0,2:49,Sometimes dogs get a little messy! Jump in the...,"[cocomelon, abckidtv, nursery rhymes, children..."
3,K4kqqCzF-BA,UCbCmjCuTUZos6Inko4u57UQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Play Outside at the Beach Song + More Nursery ...,Play Outside at the Beach Song More Nursery Rh...,2022-10-22T07:00:12Z,11744611,False,57936,0,1:1:21,The sun is shining! What a perfect beach day f...,"[cocomelon, abckidtv, nursery rhymes, children..."
4,gfZmvllWVwY,UCbCmjCuTUZos6Inko4u57UQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Halloween Song Dance! | Dance Party | CoComelo...,Halloween Song Dance Dance Party CoComelon Nur...,2022-10-18T07:00:19Z,8775011,False,46041,0,2:39,Get up and dance with all your CoComelon frien...,"[cocomelon, abckidtv, nursery rhymes, children..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5415,q-LtofpGSEs,UCchgIh8Tc4sTmBfnMQ5pDdg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Mykonos, Greece: World-Famous Beaches - Rick S...",Mykonos Greece WorldFamous Beaches Rick Steves...,2021-06-09T15:00:33Z,95956,False,1192,87,1:44,Watch the full episode: https://youtu.be/HZOuD...,"[Rick Steves, Rick Steves Europe, Europe trave..."
5416,QgaGuNenKJc,UCchgIh8Tc4sTmBfnMQ5pDdg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"""Rick Steves' Europe"" Season 11 Outtakes: The ...",Rick Steves Europe Season 11 Outtakes The Bloo...,2021-05-27T22:58:12Z,73583,False,1857,215,5:45,"Enjoy five minutes of goofy, giddy travel mome...","[Rick Steves, Rick Steves Europe, Europe trave..."
5417,cmQFp541FIM,UCchgIh8Tc4sTmBfnMQ5pDdg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Why We Travel: To Appreciate Art and Culture -...,Why We Travel To Appreciate Art and Culture Ri...,2021-05-26T15:00:15Z,49268,False,1417,84,6:48,"Part 3 of 5, this clip celebrates how travel c...","[Rick Steves, Rick Steves Europe, Europe trave..."
5418,x4JkC3QOMSc,UCchgIh8Tc4sTmBfnMQ5pDdg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Chamonix, France: Thin-Air Thrills at the Aigu...",Chamonix France ThinAir Thrills at the Aiguill...,2021-05-12T15:00:25Z,145745,False,3024,121,4:28,Watch the full episode: https://youtu.be/7HXrk...,"[Rick Steves, Rick Steves Europe, Europe trave..."


In [22]:
# export to csv
video_df.to_csv('video_df.csv', index=False)


In [21]:
# import the video_df_success.csv
video_df = pd.read_csv('video_df_success.csv')

# drop null columns
video_df = video_df.dropna(axis=1, how='all')

print(video_df.columns)
print(video_df.shape)
print(video_df.describe())


Index(['video_id', 'channel_id', 'video_title', 'video_title_clean',
       'published', 'video_views', 'video_madeforkids', 'video_likes',
       'video_comment_count', 'video_length', 'video_description',
       'video_tags'],
      dtype='object')
(5420, 12)
        video_views   video_likes  video_comment_count
count  5.420000e+03  5.420000e+03          5420.000000
mean   6.159120e+06  1.385938e+05          6369.311624
std    3.796450e+07  5.579684e+05         26269.721757
min    0.000000e+00  0.000000e+00             0.000000
25%    6.177075e+04  1.663000e+03            69.000000
50%    3.383950e+05  1.104450e+04           447.500000
75%    2.354940e+06  5.700050e+04          2363.250000
max    2.073001e+09  1.451987e+07        728578.000000


In [27]:


# convert the published column to datetime
video_df['published'] = pd.to_datetime(video_df['published'])

# convert the video_length column to datetime
#video_df['video_length'] = pd.to_datetime(video_df['video_length'])

# convert the video_madeforkids column to boolean
video_df['video_madeforkids'] = video_df['video_madeforkids'].astype(bool)

# convert the video_views column to integer
video_df['video_views'] = video_df['video_views'].astype('int64')

# convert the video_likes column to integer
video_df['video_likes'] = video_df['video_likes'].astype('int64')

# convert the video_comment_count column to integer
video_df['video_comment_count'] = video_df['video_comment_count'].astype('int64')

#check datatypes
video_df.dtypes



video_id                            object
channel_id                          object
video_title                         object
video_title_clean                   object
published              datetime64[ns, UTC]
video_views                          int64
video_madeforkids                     bool
video_likes                          int64
video_comment_count                  int64
video_length                        object
video_description                   object
video_tags                          object
dtype: object

In [28]:
# export the cleaned csv
video_df.to_csv('video_df_cleaned.csv', index=False)

In [34]:
import glob

In [35]:
# combine csv files
all_channels_df = pd.concat([pd.read_csv(f) for f in glob.glob('channels/*.csv')], ignore_index = True)

# export to csv
all_channels_df.to_csv('all_channels_df.csv', index=False)


In [ ]:
# download and save the channel thumbnail images


